# Copycat Layout via CyREST
This python notebook gives instructions and sample code for calling copycat layout through Cytoscape CyREST endpoints.

### Requirements:
1. [Install Cytoscape](http://cytoscape.org/download.php) and run the application
    1. Copycat Layout is a core app in 3.6.0 and is available upon installation. If not already installed, get it from the [App store](http://apps.cytoscape.org/apps/copycatlayout)
    
2. Verify that your CyREST port has not been changed. To check, open Cytoscape, go to `Edit>Preferences>Properties` and look for the `rest.port` property. The default port is `1234`. Set the `REST_PORT` variable below to that value
3. Install the requests python module via PyPi by running `pip install requests` in the terminal

Follow the code in this notebook to utilize the Copycat Layout via python scripts!

In [1]:
REST_PORT='1234'
REST_ENDPOINT = "http://localhost:{}/v1".format(REST_PORT)
import requests
import json

### 1. Verify that CyREST is running,
We can do this by calling the bare CyREST endpoint and checking the version.

In [2]:
response = requests.get(REST_ENDPOINT)
js_resp = response.json()
assert js_resp['apiVersion'] == 'v1', \
    "This notebook uses CyREST API v1, but version {} was found.".format(js_resp['apiVersion'])

print("CyREST v1 is running!")

CyREST v1 is running!


### 2. Load the sample session into Cytoscape
The sample.cys session file located within this repo contains 2 networks. Images of the networks will be displayed below.

Note that the `sample_modified` network is a clone of the `sample` network, with the `H` node removed, and nodes `N` and `O` appended to the network. Edges were added connecting the new nodes, and an edge from `G`->`M` was removed.

In [3]:
# Load sample session
import os
path = os.path.join(os.getcwd(), 'sample.cys')
print("Loading sample at " + path)
session_data = {'file': path}
resp = requests.get(REST_ENDPOINT + "/session", params=session_data)
assert 'message' not in resp.json(), resp.json()['message']

print("Load successful. Check Cytoscape to see two networks")

Loading sample at /Users/bsettle/git/copycat-layout/notebooks/sample.cys
Load successful. Check Cytoscape to see two networks


### 3. Get network and view SUIDs from Cytoscape
Because SUIDs are consistent among session file import/export, we could use hard coded values for the SUIDs. This code is included for the convenience of users who are not working with a consistent session file.

The first REST call gets the SUIDs of both networks, and then we use those to get the SUID of each respective network view and store them in a python dictionary

In [16]:
resp = requests.get(REST_ENDPOINT + "/networks")
network_suids = resp.json()
suid_map = {'source': {'suid': network_suids[0]}, 'target': {'suid': network_suids[1]}}

resp = requests.get("{}/networks/{}/views".format(REST_ENDPOINT, suid_map['source']['suid']))
suid_map['source']['view_suid'] = resp.json()[0] # only one view exists

resp = requests.get("{}/networks/{}/views".format(REST_ENDPOINT, suid_map['target']['suid']))
suid_map['target']['view_suid'] = resp.json()[0] # only one view exists

print(suid_map)

{'source': {'suid': 5372, 'view_suid': 5606}, 'target': {'suid': 5454, 'view_suid': 5549}}


#### EXTRA: Display network views in the notebook
The `views/{viewSUID}.png` endpoint returns an image of the network view so we can see how different our networks are.  Looking at the two images below, it would take significant time to realize that the networks are nearly identical.
Copycat Layout will fix that!

In [5]:
from IPython.display import Image, display, HTML
VIEWS = 0 # handles cached images
def show_networks():
    global VIEWS
    sourceSUID = suid_map['source']['suid']
    sourceViewSUID = suid_map['source']['view_suid']
    targetSUID = suid_map['target']['suid']
    targetViewSUID = suid_map['target']['view_suid']
    
    print("Source Network View (SUID={})".format(sourceViewSUID))
    url1 = "{}/networks/{}/views/{}.png?view={}".format(REST_ENDPOINT, sourceSUID, sourceViewSUID, VIEWS)
    im1 = Image(url=url1)
    display(im1)
    print("Target Network View (SUID={})".format(targetViewSUID))
    url2 = "{}/networks/{}/views/{}.png?view={}".format(REST_ENDPOINT, targetSUID, targetViewSUID, VIEWS)
    im2 = Image(url=url2)
    display(im2)
    
    VIEWS += 1
    
show_networks()

Source Network View (SUID=5298)


Target Network View (SUID=5241)


### 4. Apply Copycat Layout
We are going to map node locations form the source network to the target, using the `name` column in each network to pair nodes.  Upon completion, the node named 'A' in the target network will be in the same exact location as the 'A' node in the source network.  The `'selectUnmapped'` parameter is set to `True`, highlighting the differences between the two networks. Currently only unmapped nodes are highlighted.

In [15]:
# Prepare layout data
data = {
  "sourceColumn": "name",
  "targetColumn": "name",
  "selectUnmapped": True,
  "gridUnmapped": False
}
resp = requests.put("{}/apply/layouts/copycat/{}/{}".format(REST_ENDPOINT, \
            suid_map['source']['view_suid'], suid_map['target']['view_suid']), params=data)
data = resp.json()['data']

print("Mapped {} nodes, left {} unmapped in the target network".format(data['mappedNodeCount'], data['unmappedNodeCount']))

Mapped 12 nodes, left 2 unmapped in the target network


### Compare the networks now
The minor differences between the two networks are much easier to see at a glance. The unmapped nodes are highlighted, and can easily be dragged off to the side, or apply a different layout (eg GridLayout).

In [14]:
show_networks()

Source Network View (SUID=5298)


Target Network View (SUID=5241)


### Conclusion
Now that the named nodes have been aligned, we can clearly see that nodes N and O are not in the first network, and node H is not in the second.  